In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

from sklearn import metrics
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

import seaborn as sns
sns.set()

In [3]:
!dir

 Volume in drive D has no label.
 Volume Serial Number is 3417-05D0

 Directory of d:\PythonTraining\Signate\AI Quest アセスメント

2020/08/29  09:20    <DIR>          .
2020/08/29  09:20    <DIR>          ..
2020/08/29  09:14    <DIR>          .ipynb_checkpoints
2020/08/18  22:53           247,083 20200818_01.ipynb
2020/08/18  23:13           192,046 20200818_02.ipynb
2020/08/21  21:32           203,927 20200821_01.ipynb
2020/08/29  09:20            29,736 20200829_01.ipynb
2020/08/29  09:17             3,188 logs.log
2020/08/17  19:55           155,642 sample_submit.csv
2020/08/21  21:10           459,512 submit_20200818.csv
2020/08/18  22:01           297,657 submit_20200818_2.csv
2020/08/18  22:52           459,605 submit_20200818_3.csv
2020/08/18  23:09           297,668 submit_20200818_4.csv
2020/08/17  19:55        24,919,869 test.csv
2020/08/17  19:55        75,690,186 train.csv
              12 File(s)    102,956,119 bytes
               3 Dir(s)  957,316,128,768 bytes free


In [4]:
train = pd.read_csv('train.csv', index_col=0)
test = pd.read_csv('test.csv', index_col=0)
submit = pd.read_csv('sample_submit.csv', header=None)
train.shape, test.shape, submit.shape

((55583, 28), (18528, 27), (18528, 2))

In [5]:
train.head()

,accommodates,amenities,bathrooms,bed_type,bedrooms,beds,cancellation_policy,city,cleaning_fee,description,...,longitude,name,neighbourhood,number_of_reviews,property_type,review_scores_rating,room_type,thumbnail_url,zipcode,y
id,,,,,,,,,,,,,,,,,,,,,
0,6,"{TV,""Wireless Internet"",Kitchen,""Free parking ...",2.0,Real Bed,1.0,4.0,flexible,LA,t,My place is meant for family and a few friends...,...,-118.154761,The Penthouse,NaN,1,Apartment,60.0,Private room,NaN,90804,138.0
1,2,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",1.0,Real Bed,1.0,1.0,strict,DC,t,This is a new listing for a lovely guest bedro...,...,-76.978190,Guest Bedroom in Brookland,Brookland,9,House,100.0,Private room,https://a0.muscache.com/im/pictures/e4d8b51f-6...,20018,42.0
2,2,"{TV,Internet,""Wireless Internet"",Kitchen,""Indo...",2.0,Real Bed,1.0,1.0,strict,NYC,t,We're looking forward to your stay at our apt....,...,-73.926240,Clean Modern Room in Lux Apt 1 Block From J Train,Bushwick,27,Apartment,83.0,Private room,https://a0.muscache.com/im/pictures/5ffecc9b-d...,NaN,65.0
3,2,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",1.0,Real Bed,1.0,1.0,strict,SF,t,BEST CITY VIEWS - - ROOF DECK W/ BBQ & WiFi - ...,...,-122.411906,BEST views + reviews! 5/5 stars*****,Nob Hill,38,Apartment,95.0,Private room,NaN,94133,166.0
4,2,"{TV,Internet,""Wireless Internet"",""Air conditio...",1.0,Real Bed,1.0,1.0,strict,NYC,t,Charming Apartment on the upper west side of M...,...,-73.974691,Charming 1-bedroom - UWS Manhattan,Upper West Side,5,Apartment,100.0,Entire home/apt,https://a0.muscache.com/im/pictures/92879730/5...,10024,165.0


In [6]:
submit.head()

,0,1
0,0,10
1,1,10
2,2,10
3,3,10
4,4,10


In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55583 entries, 0 to 55582
Data columns (total 28 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   accommodates            55583 non-null  int64  
 1   amenities               55583 non-null  object 
 2   bathrooms               55436 non-null  float64
 3   bed_type                55583 non-null  object 
 4   bedrooms                55512 non-null  float64
 5   beds                    55487 non-null  float64
 6   cancellation_policy     55583 non-null  object 
 7   city                    55583 non-null  object 
 8   cleaning_fee            55583 non-null  object 
 9   description             55583 non-null  object 
 10  first_review            43675 non-null  object 
 11  host_has_profile_pic    55435 non-null  object 
 12  host_identity_verified  55435 non-null  object 
 13  host_response_rate      41879 non-null  object 
 14  host_since              55435 non-null

In [8]:
# check version
from pycaret.utils import version
version()

2.0


In [9]:
from pycaret.regression import *

reg1 = setup(train, target = 'y')

IntProgress(value=0, description='Processing: ', max=13)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,09:21:42
Status,. . . . . . . . . . . . . . . . . .,Preparing Data for Modeling
ETC,. . . . . . . . . . . . . . . . . .,Calculating ETC


Text(value="Following data types have been inferred automatically, if they are correct press enter to continue…

,Data Type
accommodates,Categorical
amenities,Categorical
bathrooms,Numeric
bed_type,Categorical
bedrooms,Categorical
beds,Categorical
cancellation_policy,Categorical
city,Categorical
cleaning_fee,Categorical
description,Categorical


MemoryError: Unable to allocate 88.0 GiB for an array with shape (55583, 212440) and data type float64

In [ ]:
best_model = compare_models(fold=5)

In [ ]:
import numpy as np
lgbms = [create_model('lightgbm', learning_rate=i) for i in np.arange(0.1,1,0.1)]

In [ ]:
tuned_lightgbm = tune_model(lightgbm, n_iter=50, optimize = 'RMSE')

In [ ]:
tuned_lightgbm

In [ ]:
plot_model(tuned_lightgbm)

In [ ]:
plot_model(tuned_lightgbm, plot = 'feature')

In [ ]:
interpret_model(tuned_lightgbm)

In [ ]:
submit.to_csv('submit_20200818_2.csv', index=None, header=None)